# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import json
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data/')

In [6]:
filepath = song_files[0]
filepath

'/home/ak/c2/project1/data/song_data/A/A/C/TRAACPE128F421C1B9.json'

In [7]:
df = pd.DataFrame(pd.read_json( filepath,lines=True, orient='columns'))
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARB29H41187B98F0EF,41.88415,-87.63241,Chicago,Terry Callier,SOGNCJP12A58A80271,Do You Finally Need A Friend,342.56934,1972


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id', 'title', 'artist_id', 'duration']].values.tolist()[0]
song_data

['SOGNCJP12A58A80271',
 'Do You Finally Need A Friend',
 'ARB29H41187B98F0EF',
 342.56934]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

SyntaxError: syntax error at or near "'song_id'"
LINE 2: INSERT INTO songs ('song_id', 'title', 'artist_id', 'duratio...
                           ^


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
df.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [12]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()[0]
artist_data

['ARB29H41187B98F0EF', 'Terry Callier', 'Chicago', 41.88415, -87.63241]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files('data/log_data/')

In [15]:
filepath = log_files[0]

In [16]:
df = pd.DataFrame(pd.read_json( filepath,lines=True, orient='columns'))
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
1,None,Logged In,Stefany,F,0,White,NaN,free,"Lubbock, TX",GET,Home,1540708070796,82,None,200,1541122176796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


In [17]:
##Some instruction deleted Fill it from Udacity workbook

In [18]:
df = df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
5,Dance Gavin Dance,Logged In,Marina,F,0,Sutton,218.46159,free,"Salinas, CA",PUT,NextSong,1541064343796,47,Uneasy Hearts Weigh The Most,200,1541127957796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",48


In [34]:
df['dt'] = df['ts'].apply(lambda time : pd.Timestamp(time, unit='ms'))

In [43]:
time_stamp = df.ts.values
hour = df.dt.apply(lambda x : x.hour).values
day = df.dt.apply(lambda x : x.day).values
week_of_year = df.dt.apply(lambda x : x.weekofyear).values
month = df.dt.apply(lambda x : x.month).values
year = df.dt.apply(lambda x : x.year).values
weekday = df.dt.apply(lambda x : x.day_name()).values

In [46]:
time_data = (time_stamp, hour, day, week_of_year, month,year, weekday)
column_labels = ('timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday')

In [47]:
time_df = pd.DataFrame({col:value for col, value in zip(column_labels, time_data)})
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,1541121934796,1,2,44,11,2018,Friday
1,1541122241796,1,2,44,11,2018,Friday
2,1541122457796,1,2,44,11,2018,Friday
3,1541126568796,2,2,44,11,2018,Friday
4,1541127957796,3,2,44,11,2018,Friday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [53]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId',
       'timestamp'],
      dtype='object')

In [48]:
user_df = df[['userId', 'firstName', 'lastName', 'gender']]
user_df = user_df.drop_duplicate(subset='userId')

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.